In [ ]:
import numpy as np
import cv2
import pandas as pd

def Image_Scaling(img) :
    global scale
    scale = [2, 1, 1/2, 1/4]
    scale_imgArr = []
    
    for i in scale :
        img = cv2.resize(img, (int(width*i), int(height*i)))
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        scale_imgArr.append(gray)
        
    return np.array(scale_imgArr)
                            
def make_Gaussian_Pyramid(imgArr) :
    octave = []
    k = np.sqrt(2)
    for i in range(0, len(imgArr)):
        tmp = []
        blur_level = 1 / np.sqrt(2)
        for j in range(0, 5):
            blur_level = blur_level * k
            tmp.append(cv2.GaussianBlur(imgArr[i], (5,5), blur_level))
        octave.append(tmp)
    
    return np.array(octave)

def Difference_of_Gaussian (octave) :
    img_dog = []
    for i in range(0, len(octave)):
        tmp = []
        for j in range(0, len(octave[i])-1):
            tmp.append(octave[i,j] - octave[i,j+1])
        img_dog.append(tmp)
            
    return np.array(img_dog)

def Optimization(img) :
    img_sobel_xx = cv2.Sobel(img, cv2.CV_32F, 2, 0)         ## 이미지를 x방향으로 2차 미분하기 위한 sobel_filter
    img_sobel_yy = cv2.Sobel(img, cv2.CV_32F, 0, 2)         ## 이미지를 y방향으로 2차 미분하기 위한 sobel_filter
    img_sobel_xy = cv2.Sobel(img, cv2.CV_32F, 1, 1)

    det = img_sobel_xx * img_sobel_yy - img_sobel_xy**2
    trace = img_sobel_xx + img_sobel_yy
    R = (trace**2) / det                                    ## 모든 pixel에 R값을 만듦
                                                          
    for y in range(0, R.shape[0]):                          
        for x in range(0, R.shape[1]):
            if 5 < R[y,x] or R[y,x] < 8 :                  ## threshold는 4에서 9사이
                img[y,x] = 255
            else :
                img[y,x] = 0
    return img
    
def find_key_point(img_DoG) :
    key_point = []
    for i in range(0, len(img_DoG)):
        tmp = []
        for j in range(1, len(img_DoG[i])-1):
            h, w= img_DoG[i, j].shape
            result = np.zeros((img_DoG[i,j].shape), np.uint8)
            for row in range(1, h-1):                                 ## 범위 설정은 DataFrame보고 설정을 잘해야한다. 
                for col in range(1, w-1):
                    check = img_DoG[i, j][row,col] 
                    loc_max = max(img_DoG[i,j-1][row-1:row+2 , col-1:col+2].max(), 
                                  img_DoG[i,j][row-1:row+2 , col-1:col+2].max(), 
                                  img_DoG[i,j+1][row-1:row+2 , col-1:col+2].max())
                    loc_min = min(img_DoG[i,j-1][row-1:row+2 , col-1:col+2].min(), 
                                  img_DoG[i,j][row-1:row+2 , col-1:col+2].min(), 
                                  img_DoG[i,j+1][row-1:row+2 , col-1:col+2].min())
                    
                    if check >= loc_max or check <= loc_min :
                        result[row,col] = 255
            result = Optimization(result)
            tmp.append(result)
        key_point.append(tmp)
    
    return np.array(key_point)

def get_Angle(img, x, y, scale) :
    if x <=15 or y<=15 or img.shape[0]-16 <= y or img.shape[1]-16 <= x:
        return 0
    blur_level = 1.5 * scale   ##가우시안 사이즈를 잘 조정해야한다..
    tmp = cv2.GaussianBlur(img[y-15:y+16, x-15:x+16], (31,31), blur_level)
    print(tmp, type(tmp))
    
    
    img[y-15:y+16, x-15:x+16] = tmp
    magnitude = []
    angle = []
    
    for row in range(y-15, y+16):   ## 943, 960
        for col in range(x-15, x+16):  ##1263, 1280
            magnitude.append( np.sqrt( np.square(img[row,col+1]-img[row,col-1]) + np.square(img[row+1,col] - img[row-1,col]) ))
            angle.append( np.arctan2( img[row+1,col] - img[row-1,col], img[row,col+1]-img[row,col-1] ) )
    
    magnitude = np.array(magnitude, np.uint8).reshape(tmp.shape[0], tmp.shape[1])
    angle = np.array(angle, np.uint8).reshape(tmp.shape[0], tmp.shape[1])
    
    hist1 = np.arange(0,370, 10, np.float64)
    hist2 = [0 for i in range(0,36)]
    
    for i in range(0, len(hist1)-1):
        hist1[i] = np.mean((hist1[i], hist1[i+1]-1))
    hist1 = np.delete(hist1, -1)
    
    for row in range(0, len(angle)):
        for col in range(0, len(angle[row])):
            degree = np.rad2deg(angle[row, col])
            if 0<=degree<10:      hist2[0] += magnitude[row,col]
            elif 10<=degree<20:   hist2[1] += magnitude[row,col]
            elif 20<=degree<30:   hist2[2] += magnitude[row,col]
            elif 30<=degree<40:   hist2[3] += magnitude[row,col]
            elif 40<=degree<50:   hist2[4] += magnitude[row,col]
            elif 50<=degree<60:   hist2[5] += magnitude[row,col]
            elif 60<=degree<70:   hist2[6] += magnitude[row,col]
            elif 70<=degree<80:   hist2[7] += magnitude[row,col]
            elif 80<=degree<90:   hist2[8] += magnitude[row,col]
            elif 90<=degree<100:  hist2[9] += magnitude[row,col]
            elif 100<=degree<110: hist2[10] += magnitude[row,col]
            elif 110<=degree<120: hist2[11] += magnitude[row,col]
            elif 120<=degree<130: hist2[12] += magnitude[row,col]
            elif 130<=degree<140: hist2[13] += magnitude[row,col]
            elif 140<=degree<150: hist2[14] += magnitude[row,col]
            elif 150<=degree<160: hist2[15] += magnitude[row,col]
            elif 160<=degree<170: hist2[16] += magnitude[row,col]
            elif 170<=degree<180: hist2[17] += magnitude[row,col]
            elif 180<=degree<190: hist2[18] += magnitude[row,col]
            elif 190<=degree<200: hist2[19] += magnitude[row,col]
            elif 200<=degree<210: hist2[20] += magnitude[row,col]
            elif 210<=degree<220: hist2[21] += magnitude[row,col]
            elif 220<=degree<230: hist2[22] += magnitude[row,col]
            elif 230<=degree<240: hist2[23] += magnitude[row,col]
            elif 240<=degree<250: hist2[24] += magnitude[row,col]
            elif 250<=degree<260: hist2[25] += magnitude[row,col]
            elif 260<=degree<270: hist2[26] += magnitude[row,col]
            elif 270<=degree<280: hist2[27] += magnitude[row,col]
            elif 280<=degree<290: hist2[28] += magnitude[row,col]
            elif 290<=degree<300: hist2[29] += magnitude[row,col]
            elif 300<=degree<310: hist2[30] += magnitude[row,col]
            elif 310<=degree<320: hist2[31] += magnitude[row,col]
            elif 320<=degree<330: hist2[32] += magnitude[row,col]
            elif 330<=degree<340: hist2[33] += magnitude[row,col]
            elif 340<=degree<350: hist2[34] += magnitude[row,col]
            elif 350<=degree<360: hist2[35] += magnitude[row,col]
    
    max_index = 0
    for index, value in enumerate(hist2):
        if value == max(hist2) :
            max_index = index
    
    return hist1[max_index]
    

def Orientation_Assign(imgArr) :
    info = []
    for octave in range(0, len(imgArr)):
        tmp = []
        for layer in range(0, len(imgArr[octave])):
            h, w = imgArr[octave, layer].shape
            df = pd.DataFrame(columns=['X', 'Y', 'Scale', 'Angle'])
            for row in range(0, h):
                for col in range(0,w):
                    if imgArr[octave, layer][row, col] > 0 :
                        angle = get_Angle(imgArr[octave, layer], col, row, scale[octave])
                        if angle == 0 :
                            continue
                        df = pd.concat([df, pd.DataFrame({'X': [col], 'Y':[row], 'Scale': [scale[octave]], 'Angle': [angle]})])
            tmp.append(df)
        info.append(tmp)
    return info
    
if __name__ == "__main__" :
    img = cv2.imread('opencv/picture/autoban.jpg')
    img = cv2.resize(img, (640, 480))
    
    global height, width
    height, width = img.shape[0:2]
    
    imgArr = Image_Scaling(img)
    octave = make_Gaussian_Pyramid(imgArr)    
    img_DoG = Difference_of_Gaussian(octave)
    img_kP = find_key_point(img_DoG)
    kP_info = Orientation_Assign(img_kP)
    
    cv2.imshow('a', img_kP[0,0])
    cv2.imshow('b', img_kP[1,0])
    cv2.imshow('c', img_kP[2,0])
    cv2.imshow('d', img_kP[3,0])
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [4]:
import numpy as np
import cv2
import pandas as pd

def Image_Scaling(img) :
    global scale
    scale = [2, 1, 1/2, 1/4]
    scale_imgArr = []
    
    for i in scale :
        img = cv2.resize(img, (int(width*i), int(height*i)))
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        scale_imgArr.append(gray)
        
    return np.array(scale_imgArr)
                            
def make_Gaussian_Pyramid(imgArr) :
    octave = []
    k = np.sqrt(2)
    for i in range(0, len(imgArr)):
        tmp = []
        blur_level = 1 / np.sqrt(2)
        for j in range(0, 5):
            blur_level = blur_level * k
            tmp.append(cv2.GaussianBlur(imgArr[i], (5,5), blur_level))
        octave.append(tmp)
    
    return np.array(octave)

def Difference_of_Gaussian (octave) :
    img_dog = []
    for i in range(0, len(octave)):
        tmp = []
        for j in range(0, len(octave[i])-1):
            tmp.append(octave[i,j] - octave[i,j+1])
        img_dog.append(tmp)
            
    return np.array(img_dog)

def Optimization(img) :
    img_sobel_xx = cv2.Sobel(img, cv2.CV_32F, 2, 0)         ## 이미지를 x방향으로 2차 미분하기 위한 sobel_filter
    img_sobel_yy = cv2.Sobel(img, cv2.CV_32F, 0, 2)         ## 이미지를 y방향으로 2차 미분하기 위한 sobel_filter
    img_sobel_xy = cv2.Sobel(img, cv2.CV_32F, 1, 1)

    det = img_sobel_xx * img_sobel_yy - img_sobel_xy**2
    trace = img_sobel_xx + img_sobel_yy
    R = (trace**2) / det                                    ## 모든 pixel에 R값을 만듦
                                                          
    for y in range(0, R.shape[0]):                          
        for x in range(0, R.shape[1]):
            if R[y,x] ==4 :                  ## threshold는 4에서 9사이
                img[y,x] = 255
            else :
                img[y,x] = 0
    return img
    
def find_key_point(img_DoG) :
    key_point = []
    for i in range(0, len(img_DoG)):
        tmp = []
        for j in range(1, len(img_DoG[i])-1):
            h, w= img_DoG[i, j].shape
            result = np.zeros((img_DoG[i,j].shape), np.uint8)
            for row in range(1, h-1):                                 ## 범위 설정은 DataFrame보고 설정을 잘해야한다. 
                for col in range(1, w-1):
                    check = img_DoG[i, j][row,col] 
                    loc_max = max(img_DoG[i,j-1][row-1:row+2 , col-1:col+2].max(), 
                                  img_DoG[i,j][row-1:row+2 , col-1:col+2].max(), 
                                  img_DoG[i,j+1][row-1:row+2 , col-1:col+2].max())
                    loc_min = min(img_DoG[i,j-1][row-1:row+2 , col-1:col+2].min(), 
                                  img_DoG[i,j][row-1:row+2 , col-1:col+2].min(), 
                                  img_DoG[i,j+1][row-1:row+2 , col-1:col+2].min())
                    
                    if check >= loc_max or check <= loc_min :
                        result[row,col] = 255
            result = Optimization(result)
            tmp.append(result)
        key_point.append(tmp)
    
    return np.array(key_point)

if __name__ == "__main__" :
    img = cv2.imread('opencv/picture/autoban.jpg')
    img = cv2.resize(img, (640, 480))
    
    global height, width
    height, width = img.shape[0:2]
    
    imgArr = Image_Scaling(img)
    octave = make_Gaussian_Pyramid(imgArr)    
    img_DoG = Difference_of_Gaussian(octave)
    img_kP = find_key_point(img_DoG)

    cv2.imshow('pyramid1-1', octave[1,0])
    cv2.imshow('pyramid1-2', octave[1,1])
    cv2.imshow('pyramid1-3', octave[1,2])
    cv2.imshow('pyramid1-4', octave[1,3])
    cv2.imshow('pyramid1-5', octave[1,4])
    cv2.imshow('pyramid2-1', octave[2,0])
    cv2.imshow('pyramid2-2', octave[2,1])
    cv2.imshow('pyramid2-3', octave[2,2])
    cv2.imshow('pyramid2-4', octave[2,3])
    cv2.imshow('pyramid2-5', octave[2,4])
    
    cv2.waitKey(0)
    cv2.destroyAllWindows()

C:\ProgramData\Anaconda3\envs\leejaewon\lib\site-packages\ipykernel_launcher.py:47: RuntimeWarning: divide by zero encountered in true_divide
C:\ProgramData\Anaconda3\envs\leejaewon\lib\site-packages\ipykernel_launcher.py:47: RuntimeWarning: invalid value encountered in true_divide


In [8]:
cv2.imshow('key_point1', img_kP[2,0])
cv2.imshow('key_point2', img_kP[2,1])

cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
## keypoint 찾는곳에서 이미지의 값을 255로 건드리는게 아니라 그냥 key point의 위치정보만 추출하는듯.
## 블로그에 나와있는 식을토대로 다시 진행해봐야겠다..

In [2]:
import pandas as pd
pd.DataFrame.from_records(img_kP[1,0])

,0,1,2,3,4,5,6,7,8,9,...,630,631,632,633,634,635,636,637,638,639
0,255,255,255,255,255,255,255,255,255,255,...,255,255,255,255,255,255,255,255,255,255
1,255,255,255,255,255,255,255,255,255,255,...,255,255,255,255,255,255,255,255,255,255
2,255,255,255,255,255,0,0,255,255,255,...,0,0,0,0,0,0,0,0,255,255
3,255,255,255,255,255,255,255,255,255,255,...,0,0,0,0,0,0,0,0,255,255
4,255,255,255,255,255,255,255,255,255,255,...,0,0,0,0,0,0,0,0,255,255
5,255,255,255,255,255,255,255,255,255,255,...,0,0,0,0,0,0,0,0,255,255
6,0,0,0,255,255,255,255,255,255,255,...,0,0,0,0,0,0,0,0,255,255
7,0,0,0,255,255,255,255,255,255,255,...,0,0,0,0,0,0,0,0,255,255
8,255,255,255,255,255,255,255,255,255,255,...,0,0,0,0,0,0,0,0,255,255
9,255,255,255,255,255,255,255,255,255,0,...,0,0,0,0,0,0,0,0,255,255


In [ ]:
cv2.imshow('img_kP[0,0]', img_DoG[0,0])
cv2.imshow('img_kP[0,1]', img_kP[0,1])
cv2.imshow('img_kP[1,0]', img_kP[1,0])
cv2.imshow('img_kP[1,1]', img_kP[1,1])
cv2.imshow('img_kP[2,0]', img_kP[2,0])
cv2.imshow('img_kP[2,1]', img_kP[2,1])
cv2.imshow('img_kP[3,0]', img_kP[3,0])
cv2.imshow('img_kP[3,1]', img_kP[4,1])
cv2.waitKey(0)
cv2.destroyAllWindows()

In [2]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y
